# Week13 Hands-On: LSTM Time Series Forecasting

**TensorFlow / Keras — Full version (15 epochs)**

Train an LSTM to forecast a noisy sine wave. Demonstrates windowing, scaling, LSTM model, and forecasting.

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import MinMaxScaler
import os

# Create synthetic time series (noisy sine)
def generate_series(n=2000):
    t = np.arange(n)
    series = np.sin(0.02*t) + 0.5*np.sin(0.05*t) + 0.1*np.random.randn(n)
    return series

series = generate_series(3000)
plt.figure(figsize=(10,3))
plt.plot(series[:500])
plt.title("Synthetic time series (first 500 points)")
plt.show()

# Prepare data: windowing
def make_windows(series, window_size=50, horizon=1):
    X, y = [], []
    for i in range(len(series)-window_size-horizon+1):
        X.append(series[i:i+window_size])
        y.append(series[i+window_size:i+window_size+horizon])
    return np.array(X), np.array(y)

window_size = 50
horizon = 1
X, y = make_windows(series, window_size, horizon)
y = y.reshape(-1,)

# Train/test split
split = int(len(X)*0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Scale
scaler = MinMaxScaler()
X_train_flat = X_train.reshape(-1,1)
X_test_flat = X_test.reshape(-1,1)
scaler.fit(X_train_flat)
X_train = scaler.transform(X_train_flat).reshape(X_train.shape)
X_test = scaler.transform(X_test_flat).reshape(X_test.shape)
y_train = scaler.transform(y_train.reshape(-1,1)).reshape(-1,)
y_test = scaler.transform(y_test.reshape(-1,1)).reshape(-1,)

# Build LSTM model
model = models.Sequential([
    layers.Input(shape=(window_size,1)),
    layers.LSTM(64, return_sequences=True),
    layers.LSTM(32),
    layers.Dense(16, activation='relu'),
    layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

# reshape inputs
X_train_r = X_train.reshape((-1, window_size, 1))
X_test_r = X_test.reshape((-1, window_size, 1))

# Callbacks and train
checkpoint = os.path.join("/mnt/data", "week13_hands_on_best.h5")
callbacks = [tf.keras.callbacks.ModelCheckpoint(checkpoint, save_best_only=True, monitor='val_mae'),
             tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True, monitor='val_mae')]

history = model.fit(X_train_r, y_train, epochs=15, batch_size=64,
                    validation_data=(X_test_r, y_test), callbacks=callbacks)

# Evaluate
loss, mae = model.evaluate(X_test_r, y_test, verbose=2)
print("Test MAE:", mae)

# Forecast a segment
preds = model.predict(X_test_r)
# Inverse scale
preds_inv = scaler.inverse_transform(preds)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1,1))

plt.figure(figsize=(10,4))
plt.plot(y_test_inv[:200], label='true')
plt.plot(preds_inv[:200], label='pred')
plt.legend()
plt.title("Forecast vs True (first 200 test points)")
plt.show()

# Save final model
final_model = os.path.join("/mnt/data", "week13_hands_on_model.h5")
model.save(final_model)
print("Saved model to", final_model)
